In [10]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
from ast import literal_eval
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
df = pd.read_csv('data/Topics.csv')

In [11]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [12]:
desc_blob = [TextBlob(desc) for desc in df['Description']]
df['tb_Pol'] = [b.sentiment.polarity for b in desc_blob]
df['tb_Subj'] = [b.sentiment.subjectivity for b in desc_blob]

In [17]:
df.sort_values('tb_Pol', ascending = False).iloc[0]['Description']

"On the streets of Tokyo and in meetings with some of Yasujiro Ozu's legendary collaborators, renowned director Wim Wenders explores the world of Ozu, whom Wenders considers 'a sacred treasure of cinema.'"

In [18]:
analyzer = SentimentIntensityAnalyzer()
df['compound'] = [analyzer.polarity_scores(v)['compound'] for v in df['Description']]
df['neg'] = [analyzer.polarity_scores(v)['neg'] for v in df['Description']]
df['neu'] = [analyzer.polarity_scores(v)['neu'] for v in df['Description']]
df['pos'] = [analyzer.polarity_scores(v)['pos'] for v in df['Description']]

In [23]:
df.sort_values('pos', ascending = False).iloc[0]['Description']

'One of the great translations of literature into film, David Lean’s GREAT EXPECTATIONS brings Charles Dickens’s masterpiece to robust on-screen life. Pip, Magwitch, Miss Havisham, and Estella populate Lean’s magnificent miniature, beautifully photographed by Guy Green and designed by John Bryan.'

In [24]:
tfidf = TfidfVectorizer(stop_words='english' )
tfidf_matrix = tfidf.fit_transform(df['Description'])
tfidf_matrix.shape

(1436, 16196)

In [25]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(1436, 1436)

In [34]:
indices = pd.Series(df.index, index=df['Title']).drop_duplicates()
def get_recommendations(title,cosine_sim = cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df['Title'].iloc[movie_indices]

In [35]:
#Now to get recommendations one will input the movie title and our recommender will generate top 10 movies similar to the input movie based on similar plots or stories¶
get_recommendations('Seven Samurai')

345                          Drunken Angel
1077                               Sanjuro
1420                               Yojimbo
1021                             Red Beard
1073                          Samurai Saga
1070    Samurai III: Duel at Ganryu Island
68                      The Bad Sleep Well
643                                  Kill!
1071           Samurai I: Musashi Miyamoto
1088                               Scandal
Name: Title, dtype: object

In [ ]:
#trying to merge rating data frame for collaborative filtering

In [99]:
ratings = pd.read_csv("ratings.csv")
movies = pd.read_csv("movie.csv")

dataset = pd.merge(movies, ratings, how="left" , on=['movieId'])
dataset = dataset.loc[:,["userId","movieId","Title","Description","rating"]]
df_ratings = dataset.loc[:,["title","rating"]].groupby("title").mean()
genres = dataset["Description"]
dataset

,userId,movieId,Title,Description,rating
0,2.0,1,2 or 3 Things I Know About Her,In 2 OR 3 THINGS I KNOW ABOUT HER (2 OU 3 CHOS...,5.0
1,5.0,1,2 or 3 Things I Know About Her,In 2 OR 3 THINGS I KNOW ABOUT HER (2 OU 3 CHOS...,4.0
2,8.0,1,2 or 3 Things I Know About Her,In 2 OR 3 THINGS I KNOW ABOUT HER (2 OU 3 CHOS...,5.0
3,11.0,1,2 or 3 Things I Know About Her,In 2 OR 3 THINGS I KNOW ABOUT HER (2 OU 3 CHOS...,4.0
4,14.0,1,2 or 3 Things I Know About Her,In 2 OR 3 THINGS I KNOW ABOUT HER (2 OU 3 CHOS...,4.0
...,...,...,...,...,...
40546,607.0,1620,Zombi Child,Iconoclastic auteur Bertrand Bonello blends vo...,3.0
40547,615.0,1620,Zombi Child,Iconoclastic auteur Bertrand Bonello blends vo...,4.0
40548,627.0,1620,Zombi Child,Iconoclastic auteur Bertrand Bonello blends vo...,3.0
40549,662.0,1620,Zombi Child,Iconoclastic auteur Bertrand Bonello blends vo...,4.0


In [100]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
li = []
for i in range(len(genres)):
    temp = genres[i].split(" ")
    for j in range(len(temp)):
        temp[j] = lemmatizer.lemmatize(temp[j])
    li.append(" ".join(temp))
print(li)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [101]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
X = cv.fit_transform(li).toarray()

genres = pd.DataFrame(X,columns=cv.get_feature_names())

dataset = dataset.iloc[:,:-2]
new_dataset = dataset.join(genres)

In [97]:
new_dataset


,userId,movieId,Title,000,10,100,1000,1080,11,125th,...,zouzou,zsigmond,zsófia,zwerin,ágnes,ángel,åke,élan,émigré,šerbedžija
0,2.0,1,2 or 3 Things I Know About Her,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5.0,1,2 or 3 Things I Know About Her,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8.0,1,2 or 3 Things I Know About Her,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11.0,1,2 or 3 Things I Know About Her,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14.0,1,2 or 3 Things I Know About Her,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40546,607.0,1620,Zombi Child,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40547,615.0,1620,Zombi Child,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40548,627.0,1620,Zombi Child,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40549,662.0,1620,Zombi Child,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
